In [1]:
import pandas as pd
import numpy as np

In [54]:
df_test = pd.read_csv(r'C:\Users\racco\Documents\DataScience\Python_Datasets\test.csv')
df_train = pd.read_csv(r'C:\Users\racco\Documents\DataScience\Python_Datasets\train.csv')

In [3]:
print(df_train.shape)
df_train.columns

(891, 12)


Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
print(df_test.shape)
df_test.columns

(418, 11)


Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:
#df_train.isnull().sum() # Age、Cabin needs Imputation
df_train['Cabin'].isnull().sum()

687

In [6]:
'''人數相關'''
df_train['fam_size'] = df_train['SibSp'] + df_train['Parch'] + 1
df_train['Alone'] = (df_train['fam_size']==1)
df_train['Alone'] = df_train['Alone'].astype(int)
'''種類'''
df_train['Deck'] = df_train['Cabin'].str.slice(0, 1)
df_train['Deck'] = pd.Categorical(df_train['Deck'], 
                                  categories = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', 'Z'])
'''人名/稱謂'''
df_train['Title'] = df_train.Name.str.split(', ').str[1].str.split('.').str[0].str.strip()
df_train['Title'] = df_train['Title'].replace('Ms','Miss')
df_train['Title'] = df_train['Title'].replace('Mlle','Miss')
df_train['Title'] = df_train['Title'].replace('Mme','Mrs')
title_mask = (df_train['Title'] != 'Mr') | (df_train['Title'] != 'Miss') #| (df_train['Title'] != 'Mrs')
df_train.loc[title_mask, 'Title'] = 'Titled'
df_train['Embarked'] = df_train['Embarked'].fillna('S')
df_train['Embarked'] = df_train['Embarked'].astype(str)
'''狀態'''
df_train['Married_F'] = (df_train['Title'] != 'Miss') & (df_train['Sex'] == 'female')
df_train['Married_F'] = df_train['Married_F'].astype(int)
df_train['fam_Class'] = df_train['fam_size'] * df_train['Pclass']
df_train['fare_perp'] = df_train['Fare']/df_train['fam_size']

In [7]:
df_new = pd.get_dummies(df_train, columns = ['Embarked', 'Sex', 'Title'], dummy_na = False) # , 'Classed_Age'
df_new = df_new.drop(['Cabin', 'PassengerId', 'Ticket', 'Name', 'Deck'], axis = 1)
print(df_new.shape, '\n', df_new.columns)

(891, 17) 
 Index(['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'fam_size',
       'Alone', 'Married_F', 'fam_Class', 'fare_perp', 'Embarked_C',
       'Embarked_Q', 'Embarked_S', 'Sex_female', 'Sex_male', 'Title_Titled'],
      dtype='object')


In [8]:
df_new.isnull().sum() # Age

Survived          0
Pclass            0
Age             177
SibSp             0
Parch             0
Fare              0
fam_size          0
Alone             0
Married_F         0
fam_Class         0
fare_perp         0
Embarked_C        0
Embarked_Q        0
Embarked_S        0
Sex_female        0
Sex_male          0
Title_Titled      0
dtype: int64

## Impute Age with KNN Method

In [9]:
from sklearn.neighbors import KNeighborsRegressor
x = df_new.drop('Survived', axis = 1)
age_na_x = x[x['Age'].isna()].drop('Age', axis = 1)
age_nna_x = x[~x['Age'].isna()].drop('Age', axis = 1)
age_nna_y = x[~x['Age'].isna()]['Age']
age_imp = KNeighborsRegressor(n_neighbors = 10)
age_imp.fit(np.array(age_nna_x), np.array(age_nna_y))
age_pred = age_imp.predict(np.array(age_na_x))

In [37]:
#age_pred.tolist()

In [10]:
df_new.loc[df_new['Age'].isnull(), 'Age'] = age_pred

In [11]:
df_new['Age_M'] = df_new['Age']*df_new['Sex_male']
df_new['Age_F'] = df_new['Age']*df_new['Sex_female']
df_new['M_Alone'] = df_new['Alone']*df_new['Sex_male']
df_new['F_Alone'] = df_new['Alone']*df_new['Sex_female']

In [43]:
#df_new.isnull().sum()

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, classification_report
Y = df_new['Survived']
X = df_new.drop(['Survived'], axis = 1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)
clf = KNeighborsClassifier(n_neighbors = 50)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)

print("Accuracy: ", accuracy_score(Y_test, Y_pred))
print("Precision: ", precision_score(Y_test, Y_pred))
print("Confusion Matrix:\n", confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

Accuracy:  0.8100558659217877
Precision:  0.7464788732394366
Confusion Matrix:
 [[92 18]
 [16 53]]
             precision    recall  f1-score   support

          0       0.85      0.84      0.84       110
          1       0.75      0.77      0.76        69

avg / total       0.81      0.81      0.81       179



In [51]:
df_new.columns

Index(['PassengerId', 'Pclass', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'fam_size', 'Alone', 'Deck', 'Married_F', 'fam_Class',
       'fare_perp', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Sex_female',
       'Sex_male', 'Title_Titled'],
      dtype='object')

In [13]:
from sklearn.svm import SVC

svclassifier = SVC() #kernel = 'poly')
svclassifier.fit(X, Y)
Y_pred = svclassifier.predict(X_test)

In [14]:
print("Accuracy: ", accuracy_score(Y_test, Y_pred))
print("Precision: ", precision_score(Y_test, Y_pred))
print("Confusion Matrix:\n", confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))


Accuracy:  0.9329608938547486
Precision:  0.9253731343283582
Confusion Matrix:
 [[105   5]
 [  7  62]]
             precision    recall  f1-score   support

          0       0.94      0.95      0.95       110
          1       0.93      0.90      0.91        69

avg / total       0.93      0.93      0.93       179



In [16]:
df_test.head(1)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q


In [55]:
df_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [56]:
'''人數相關'''
df_test['fam_size'] = df_test['SibSp'] + df_test['Parch'] + 1
df_test['Alone'] = (df_test['fam_size']==1)
df_test['Alone'] = df_test['Alone'].astype(int)
'''種類'''
df_test['Deck'] = df_test['Cabin'].str.slice(0, 1)
df_test['Deck'] = pd.Categorical(df_test['Deck'], 
                                  categories = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', 'Z'])
'''人名/稱謂'''
df_test['Title'] = df_test.Name.str.split(', ').str[1].str.split('.').str[0].str.strip()
df_test['Title'] = df_test['Title'].replace('Ms','Miss')
df_test['Title'] = df_test['Title'].replace('Mlle','Miss')
df_test['Title'] = df_test['Title'].replace('Mme','Mrs')
title_mask = (df_test['Title'] != 'Mr') | (df_test['Title'] != 'Miss') #| (df_train['Title'] != 'Mrs')
df_test.loc[title_mask, 'Title'] = 'Titled'
df_test['Embarked'] = df_test['Embarked'].fillna('S')
df_test['Embarked'] = df_test['Embarked'].astype(str)
'''狀態'''
df_test['Married_F'] = (df_test['Title'] != 'Miss') & (df_test['Sex'] == 'female')
df_test['Married_F'] = df_test['Married_F'].astype(int)
df_test['fam_Class'] = df_test['fam_size'] * df_test['Pclass']
df_test['fare_perp'] = df_test['Fare']/df_test['fam_size']

In [41]:
#test_age_nna_x.isnull().sum()

In [57]:
df_test = pd.get_dummies(df_test, columns = ['Embarked', 'Sex', 'Title'], dummy_na = False)
test_imp_age = df_test.drop(['Cabin', 'PassengerId', 'Ticket', 'Name', 'Deck', 'Fare', 'fare_perp'], 
                            axis = 1)
test_age_na_x = test_imp_age[test_imp_age['Age'].isna()].drop('Age', axis = 1)
test_age_nna_x = test_imp_age[~test_imp_age['Age'].isna()].drop('Age', axis = 1)
test_age_nna_y = test_imp_age[~test_imp_age['Age'].isna()]['Age']
test_age_imp = KNeighborsRegressor(n_neighbors = 10)
test_age_imp.fit(np.array(test_age_nna_x), np.array(test_age_nna_y))
test_age_pred = test_age_imp.predict(np.array(test_age_na_x))
df_test.loc[df_test['Age'].isnull(), 'Age'] = test_age_pred
df_test['Age_M'] = df_test['Age']*df_test['Sex_male']
df_test['Age_F'] = df_test['Age']*df_test['Sex_female']
df_test['M_Alone'] = df_test['Alone']*df_test['Sex_male']
df_test['F_Alone'] = df_test['Alone']*df_test['Sex_female']

In [58]:
test_imp_cab = df_test.drop(['PassengerId', 'Ticket', 'Name', 'Deck', 'Fare', 'fare_perp'], 
                            axis = 1)
test_cab_na_x = test_imp_cab[test_imp_cab['Cabin'].isna()].drop('Cabin', axis = 1)
test_cab_nna_x = test_imp_cab[~test_imp_cab['Cabin'].isna()].drop('Cabin', axis = 1)
test_cab_nna_y = test_imp_cab[~test_imp_cab['Cabin'].isna()]['Cabin']
test_cab_imp = KNeighborsClassifier(n_neighbors = 50)
test_cab_imp.fit(np.array(test_cab_nna_x), np.array(test_cab_nna_y))
test_cab_pred = test_cab_imp.predict(np.array(test_cab_na_x))
df_test.loc[df_test['Cabin'].isnull(), 'Cabin'] = test_cab_pred

In [52]:
'''df_test = pd.get_dummies(df_test, columns = ['Cabin'], dummy_na = False)
test_imp_dek = df_test.drop(['PassengerId', 'Ticket', 'Name', 'Fare', 'fare_perp'], 
                            axis = 1)
test_dek_na_x = test_imp_dek[test_imp_dek['Deck'].isna()].drop('Deck', axis = 1)
test_dek_nna_x = test_imp_dek[~test_imp_dek['Deck'].isna()].drop('Deck', axis = 1)
test_dek_nna_y = test_imp_dek[~test_imp_dek['Deck'].isna()]['Deck']
test_dek_imp = KNeighborsClassifier(n_neighbors = 50)
test_dek_imp.fit(np.array(test_dek_nna_x), np.array(test_dek_nna_y))
test_dek_pred = test_dek_imp.predict(np.array(test_dek_na_x))
df_test.loc[df_test['Deck'].isnull(), 'Deck'] = test_dek_pred'''

In [59]:
df_test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'fam_size', 'Alone', 'Deck', 'Married_F', 'fam_Class',
       'fare_perp', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Sex_female',
       'Sex_male', 'Title_Titled', 'Age_M', 'Age_F', 'M_Alone', 'F_Alone'],
      dtype='object')

In [62]:
test_imp_fare.columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'fam_size', 'Alone', 'Deck',
       'Married_F', 'fam_Class', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
       'Sex_female', 'Sex_male', 'Title_Titled', 'Age_M', 'Age_F', 'M_Alone',
       'F_Alone'],
      dtype='object')

In [64]:
test_imp_fare = df_test.drop(['PassengerId', 'Ticket', 'Name', 'fare_perp', 'Deck', 'Cabin'], 
                            axis = 1)
test_fare_na_x = test_imp_fare[test_imp_fare['Fare'].isna()].drop('Fare', axis = 1)
test_fare_nna_x = test_imp_fare[~test_imp_fare['Fare'].isna()].drop('Fare', axis = 1)
test_fare_nna_y = test_imp_fare[~test_imp_fare['Fare'].isna()]['Fare']
test_fare_imp = KNeighborsClassifier(n_neighbors = 50)
test_fare_imp.fit(np.array(test_fare_nna_x), np.array(test_fare_nna_y))
test_fare_pred = test_fare_imp.predict(np.array(test_fare_na_x))
df_test.loc[df_test['Fare'].isnull(), 'Fare'] = test_fare_pred

ValueError: Unknown label type: 'continuous'

In [72]:
a = "1"
b = 2
try:
    print(a + b)
except Exception as e:
    print(e)

must be str, not int
